<a href="https://colab.research.google.com/github/tsm-mehmetakiftasoz/tsm_makif/blob/main/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D0%B8%20%D0%BE%D1%82%D1%87%D1%91%D1%82%20%D0%BF%D0%BE%20%D1%81%D0%BF%D0%B5%D1%86%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F%D0%BC%20HILTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HİLTİ Speklerini incelemek için oluşturuldu. Spek excellerini bir dosyada topla program sizden dosyadan excelleri seçmenizi isteyecek sonra her bir speki tek tek bir structured halde excellere kaydecek ayrıca overall bir df oluşturacak sonra bu df i excele çevirip ekleyecek.  hatalar olduğu zaman program içinde görünecektir ayrıca rapor için bir excel dosyası oluşturup bu hataları görmeni sağlayacak. Raporda oluşturulan ve hata alınan specler ayrıca hatalar  gözükecek /
Программа была создана для анализа спецификаций HILTI. Соберите Excel-файлы со спецификациями в одной папке — программа попросит вас выбрать эти файлы. Затем она поочерёдно сохранит каждую спецификацию в структурированном виде в Excel, а также сформирует общий DataFrame, который также будет преобразован в Excel и добавлен.
Ошибки, возникшие в процессе, будут отображаться внутри программы. Кроме того, будет создан отдельный Excel-файл для отчёта, в котором вы сможете увидеть эти ошибки. В отчёте будут отображены как успешно обработанные спецификации, так и те, в которых возникли ошибки, а также сами ошибки.

In [8]:
from google.colab import files
import pandas as pd
import os
import re

In [11]:
# Excel dosyalarını yükleyin
uploaded = files.upload()

Saving Kopya _HCN-230154 от 21.04.2025.xlsx to Kopya _HCN-230154 от 21.04.2025.xlsx
Saving Kopya _HCN-230159 от 11.06.2025.xlsx to Kopya _HCN-230159 от 11.06.2025.xlsx
Saving Kopya _HM-30085 от 29.04.2025.xlsx to Kopya _HM-30085 от 29.04.2025.xlsx
Saving Kopya SP HE-20007  to TSM-07-22-898.xlsx to Kopya SP HE-20007  to TSM-07-22-898.xlsx
Saving Kopya SP HM-to TSM-07-23-337 с переводом.xlsx to Kopya SP HM-to TSM-07-23-337 с переводом.xlsx


In [12]:
# Tüm temiz veriler burada birikecek
master_df = pd.DataFrame()

In [16]:
# Durum raporu için tablo
rapor_kayitlari = []

In [6]:
silinecek
#bunu sileceğiz
for filename in uploaded.keys():
    print(f"İşleniyor: {filename}")

    try:
        # Dosyayı tüm hücreleri string olarak oku, birleşik hücre sorunlarını önlemek için header=None
        df_raw = pd.read_excel(filename, header=None, dtype=str)

        # Başlangıç ve bitiş satırlarını tespit et
        start_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Спецификация №").any(), axis=1)].index
        end_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index

        if len(start_idx) == 0 or len(end_idx) == 0:
            print(f"Uyarı: {filename} içinde başlangıç veya bitiş satırı bulunamadı.")
            continue

        s = start_idx[0] + 1  # başlıktan sonraki satır
        e = end_idx[0]        # bitiş satırı dahil edilmez
        df_table = df_raw.iloc[s:e].copy()

        # Temizlik: boş satırları at, ilk satırı başlık yap
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Temizlenmiş veriyi kaydet
        out_filename = f"cleaned_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"{out_filename} dosyası oluşturuldu.")

    except Exception as e:
        print(f"Hata oluştu ({filename}): {e}")

İşleniyor: Kopya _HCN-230154 от 21.04.2025.xlsx
cleaned_Kopya _HCN-230154 от 21.04.2025.xlsx dosyası oluşturuldu.


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [10]:
silinecek
#bunu da sileceğiz
for filename in uploaded.keys():
    print(f"\nİşleniyor/Обработка: {filename}")

    try:
        # Excel içeriğini oku (tüm hücreleri string gibi)
        df_raw = pd.read_excel(filename, header=None, dtype=str)

        # Başlangıç ve bitiş indekslerini bul
        start_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Спецификация №").any(), axis=1)].index
        end_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index

        if len(start_idx) == 0 or len(end_idx) == 0:
            print("Spesifikasyon veya fiyat sonu satırı bulunamadı./Спецификация или конечная цена не найдены. ")
            continue

        # Başlık içeren satırı yakala
        spec_row = df_raw.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        # Spesifikasyon numarasını çek
        match_spec = re.search(r'Спецификация №\s+(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor/неизвестный'

        # Tarihi çek
        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor/неизвестный'

        # Structured tabloyu ayıkla
        s = start_idx[0] + 1
        e = end_idx[0]
        df_table = df_raw.iloc[s:e].copy()

        # Temizlik
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Spesifikasyon bilgilerini ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Yeni dosya olarak kaydet
        out_filename = f"structured_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"{out_filename} oluşturuldu/Готово ✔")

    except Exception as e:
        print(f"Hata ({filename}): {e}")


İşleniyor/Обработка: Kopya _HCN-230154 от 21.04.2025.xlsx
structured_Kopya _HCN-230154 от 21.04.2025.xlsx oluşturuldu/Готово ✔


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [13]:
 silinecek
 #Her dosyayı sırayla işle
for filename in uploaded.keys():
    print(f"\nİşleniyor/Обработка: {filename}")

    try:
        df_raw = pd.read_excel(filename, header=None, dtype=str)

        # Başlangıç ve bitiş indekslerini bul
        start_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Спецификация №").any(), axis=1)].index
        end_idx = df_raw[df_raw.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index

        if len(start_idx) == 0 or len(end_idx) == 0:
            print("Spesifikasyon veya fiyat sonu satırı bulunamadı./Спецификация или конечная цена не найдены.")
            continue

        # Başlık içeren satır
        spec_row = df_raw.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        # Spesifikasyon no
        match_spec = re.search(r'Спецификация №\s+(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor/неизвестный'

        # Tarih
        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor/неизвестный'

        # Structured tablo
        s = start_idx[0] + 1
        e = end_idx[0]
        df_table = df_raw.iloc[s:e].copy()

        # Temizle
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Spec bilgilerini ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Her bir dosyanın çıktısını kaydet
        out_filename = f"structured_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"{out_filename} oluşturuldu/Готово ✔")

        # Ana tabloya birleştir (farklı sütunlar varsa dıştan birleştir)
        master_df = pd.concat([master_df, df_table], axis=0, ignore_index=True, sort=False)

    except Exception as e:
        print(f"Hata ({filename}): {e}")


İşleniyor/Обработка: Kopya _HCN-230154 от 21.04.2025.xlsx
structured_Kopya _HCN-230154 от 21.04.2025.xlsx oluşturuldu/Готово ✔

İşleniyor/Обработка: Kopya _HCN-230159 от 11.06.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


structured_Kopya _HCN-230159 от 11.06.2025.xlsx oluşturuldu/Готово ✔
Hata (Kopya _HCN-230159 от 11.06.2025.xlsx): Reindexing only valid with uniquely valued Index objects

İşleniyor/Обработка: Kopya _HM-30085 от 29.04.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


structured_Kopya _HM-30085 от 29.04.2025.xlsx oluşturuldu/Готово ✔

İşleniyor/Обработка: Kopya SP HE-20007  to TSM-07-22-898.xlsx
Spesifikasyon veya fiyat sonu satırı bulunamadı./Спецификация или конечная цена не найдены.

İşleniyor/Обработка: Kopya SP HM-to TSM-07-23-337 с переводом.xlsx
structured_Kopya SP HM-to TSM-07-23-337 с переводом.xlsx oluşturuldu/Готово ✔


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [17]:
silinecek #
for filename in uploaded.keys():
    print(f"\n🔄 İşleniyor: {filename}")

    try:
        # Çok sayfalı Excel dosyasını aç
        xls = pd.ExcelFile(filename)

        # Başlangıç için ilk sayfa
        df_start = pd.read_excel(xls, sheet_name=0, header=None, dtype=str)

        # Başlangıç indeksini bul ("Спецификация №")
        start_idx = df_start[df_start.apply(lambda row: row.astype(str).str.contains("Спецификация №").any(), axis=1)].index
        if len(start_idx) == 0:
            print("❌ Спецификация № bulunamadı, dosya atlandı.")
            continue

        # Başlık satırı
        spec_row = df_start.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        # Spesifikasyon no
        match_spec = re.search(r'Спецификация №\s+(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor/неизвестный'

        # Tarih
        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor/неизвестный'

        # "Цена без НДС" içeren satırı tüm sayfalarda ara
        end_found = False
        for sheet in xls.sheet_names:
            df_check = pd.read_excel(xls, sheet_name=sheet, header=None, dtype=str)
            end_idx = df_check[df_check.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index

            if len(end_idx) > 0:
                end_sheet = sheet
                end_index = end_idx[0]
                end_found = True
                break

        if not end_found:
            print("❌ 'Цена без НДС' bulunamadı, dosya atlandı.")
            continue

        # Structured veri alanı: sayfa atlamalıysa birleştir
        if end_sheet == xls.sheet_names[0]:
            df_table = df_start.iloc[start_idx[0]+1:end_index].copy()
        else:
            df1 = df_start.iloc[start_idx[0]+1:].copy()
            df2 = pd.read_excel(xls, sheet_name=end_sheet, header=None, dtype=str).iloc[:end_index].copy()
            df_table = pd.concat([df1, df2], ignore_index=True)

        # Temizlik
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Spec bilgilerini ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Temiz dosyayı kaydet
        out_filename = f"cleaned_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"✅ {out_filename} oluşturuldu.")

        # Ana tabloya ekle (farklı kolonları dıştan birleştir)
        master_df = pd.concat([master_df, df_table], axis=0, ignore_index=True, sort=False)

    except Exception as e:
        print(f"⚠️ Hata ({filename}): {e}")


🔄 İşleniyor: Kopya _HCN-230154 от 21.04.2025.xlsx
✅ cleaned_Kopya _HCN-230154 от 21.04.2025.xlsx oluşturuldu.

🔄 İşleniyor: Kopya _HCN-230159 от 11.06.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned_Kopya _HCN-230159 от 11.06.2025.xlsx oluşturuldu.
⚠️ Hata (Kopya _HCN-230159 от 11.06.2025.xlsx): Reindexing only valid with uniquely valued Index objects

🔄 İşleniyor: Kopya _HM-30085 от 29.04.2025.xlsx
✅ cleaned_Kopya _HM-30085 от 29.04.2025.xlsx oluşturuldu.

🔄 İşleniyor: Kopya SP HE-20007  to TSM-07-22-898.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


❌ Спецификация № bulunamadı, dosya atlandı.

🔄 İşleniyor: Kopya SP HM-to TSM-07-23-337 с переводом.xlsx
✅ cleaned_Kopya SP HM-to TSM-07-23-337 с переводом.xlsx oluşturuldu.


In [18]:
# Her dosyayı sırayla işle
for filename in uploaded.keys():
    print(f"\n🔄 İşleniyor: {filename}")

    try:
        xls = pd.ExcelFile(filename)

        # Başlangıç verisi ilk sayfadan
        df_start = pd.read_excel(xls, sheet_name=0, header=None, dtype=str)
        start_idx = df_start[df_start.apply(lambda row: row.astype(str).str.contains("Спецификация №").any(), axis=1)].index
        if len(start_idx) == 0:
            mesaj = "Спецификация № bulunamadı ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya": filename,
                "Спецификация №": "Yok",
                "Tarih": "Yok",
                "Durum": mesaj
            })
            continue

        # Spesifikasyon bilgisi
        spec_row = df_start.iloc[start_idx[0]].astype(str).tolist()
        spec_text = ' '.join(spec_row)

        match_spec = re.search(r'Спецификация №\s+(\S+)', spec_text)
        spec_number = match_spec.group(1) if match_spec else 'Bilinmiyor'

        match_date = re.search(r'от\s+(\d{2}\.\d{2}\.\d{4})', spec_text)
        spec_date = match_date.group(1) if match_date else 'Bilinmiyor'

        # "Цена без НДС" araması tüm sayfalarda
        end_found = False
        for sheet in xls.sheet_names:
            df_check = pd.read_excel(xls, sheet_name=sheet, header=None, dtype=str)
            end_idx = df_check[df_check.apply(lambda row: row.astype(str).str.contains("Цена без НДС").any(), axis=1)].index
            if len(end_idx) > 0:
                end_sheet = sheet
                end_index = end_idx[0]
                end_found = True
                break

        if not end_found:
            mesaj = "'Цена без НДС' bulunamadı ❌"
            print(mesaj)
            rapor_kayitlari.append({
                "Dosya": filename,
                "Спецификация №": spec_number,
                "Tarih": spec_date,
                "Durum": mesaj
            })
            continue

        # Structured tabloyu oluştur
        if end_sheet == xls.sheet_names[0]:
            df_table = df_start.iloc[start_idx[0]+1:end_index].copy()
        else:
            df1 = df_start.iloc[start_idx[0]+1:].copy()
            df2 = pd.read_excel(xls, sheet_name=end_sheet, header=None, dtype=str).iloc[:end_index].copy()
            df_table = pd.concat([df1, df2], ignore_index=True)

        # Temizle
        df_table = df_table.dropna(how='all')
        df_table.columns = df_table.iloc[0]
        df_table = df_table[1:]

        # Bilgileri ekle
        df_table["номер спецификации"] = spec_number
        df_table["дата"] = spec_date

        # Kaydet
        out_filename = f"cleaned_{filename}"
        df_table.to_excel(out_filename, index=False)
        print(f"✅ {out_filename} oluşturuldu.")

        # Ana tabloya ekle
        master_df = pd.concat([master_df, df_table], axis=0, ignore_index=True, sort=False)

        # Rapor tablosuna kayıt
        rapor_kayitlari.append({
            "Dosya": filename,
            "Спецификация №": spec_number,
            "Tarih": spec_date,
            "Durum": "Oluşturuldu ✅"
        })

    except Exception as e:
        hata_mesaji = f"Hata: {str(e)}"
        print(f"⚠️ {hata_mesaji}")
        rapor_kayitlari.append({
            "Dosya": filename,
            "Спецификация №": "Bilinmiyor",
            "Tarih": "Bilinmiyor",
            "Durum": hata_mesaji
        })


🔄 İşleniyor: Kopya _HCN-230154 от 21.04.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned_Kopya _HCN-230154 от 21.04.2025.xlsx oluşturuldu.

🔄 İşleniyor: Kopya _HCN-230159 от 11.06.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned_Kopya _HCN-230159 от 11.06.2025.xlsx oluşturuldu.
⚠️ Hata: Reindexing only valid with uniquely valued Index objects

🔄 İşleniyor: Kopya _HM-30085 от 29.04.2025.xlsx


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


✅ cleaned_Kopya _HM-30085 от 29.04.2025.xlsx oluşturuldu.

🔄 İşleniyor: Kopya SP HE-20007  to TSM-07-22-898.xlsx
Спецификация № bulunamadı ❌

🔄 İşleniyor: Kopya SP HM-to TSM-07-23-337 с переводом.xlsx
✅ cleaned_Kopya SP HM-to TSM-07-23-337 с переводом.xlsx oluşturuldu.


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [22]:
# Tüm verileri kaydet
master_out = "merged_all_cleaned.xlsx"
master_df.to_excel(master_out, index=False)
print(f"\n📁 Tüm veriler {master_out} dosyasında toplandı.")
files.download(master_out)


📁 Tüm veriler merged_all_cleaned.xlsx dosyasında toplandı.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [23]:
# Rapor dosyasını oluştur
rapor_df = pd.DataFrame(rapor_kayitlari)
rapor_out = "rapor_durum.xlsx"
rapor_df.to_excel(rapor_out, index=False)
print(f"📋 Rapor {rapor_out} dosyasına kaydedildi.")
files.download(rapor_out)

📋 Rapor rapor_durum.xlsx dosyasına kaydedildi.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
silebiliriz
# ✅ Tüm verileri birleştirdikten sonra tek bir dosyada kaydet
master_out = "merged_all_cleaned.xlsx"
master_df.to_excel(master_out, index=False)
print(f"\n🔗 Bütün veriler {master_out} dosyasına kaydedildi.")

In [ ]:
silebiliriz
# Son çıktıyı indirilebilir yap
files.download(master_out)

In [24]:
for file in os.listdir():
    if file.startswith("structured_") and file.endswith(".xlsx"):
        print(f"İndiriliyor/Загрузка: {file}")
        files.download(file)

İndiriliyor/Загрузка: structured_Kopya _HM-30085 от 29.04.2025.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

İndiriliyor/Загрузка: structured_Kopya _HCN-230159 от 11.06.2025.xlsx


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

İndiriliyor/Загрузка: structured_Kopya _HCN-230154 от 21.04.2025.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>